<a href="https://colab.research.google.com/github/BlackBoyZeus/VisionNetwork/blob/main/action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.8 torchvision
!pip install -U openmim
!mim install mmengine
!mim install mmcv
!mim install mmdet
!mim install mmpose


ERROR: Could not find a version that satisfies the requirement torch==1.8 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.8
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
  Using cached mmengine-0.8.4-py3-none-any.whl (437 kB)
  Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
  Using cached yapf-0.40.1-py3-none-any.whl (250 kB)
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
  Using cached https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/mmcv-2.0.1-cp310-cp310-manylinux1_x86_64.whl (74.4 MB)
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.9 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00
     ━

In [2]:
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2
!pip install -v -e .


Cloning into 'mmaction2'...
remote: Enumerating objects: 22767, done.
remote: Counting objects: 100% (1845/1845), done.
remote: Compressing objects: 100% (1208/1208), done.
remote: Total 22767 (delta 962), reused 1253 (delta 612), pack-reused 20922
Receiving objects: 100% (22767/22767), 70.06 MiB | 16.20 MiB/s, done.
Resolving deltas: 100% (15791/15791), done.
/content/mmaction2
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Obtaining file:///content/mmaction2
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-i8j61jr9/mmaction2.egg-info
  writing manifest file '/tmp/pip-pip-egg-info-i8j61jr9/mmaction2.egg-info/SOURCES.txt'
  writing manifest file '/tmp/pip-pip-egg-info-i8j61jr9/mmaction2.egg-info/SOURCES.txt'
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.2 MB/s eta 0:00:00
 

In [3]:
!pip install mmaction2


In [4]:
!mim download mmaction2 --config tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb --dest .


processing tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 MiB 30.9 MB/s eta 0:00:00
Successfully downloaded tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth to /content/mmaction2
Successfully dumped tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb.py to /content/mmaction2


In [7]:
# Google Colab cell for inference on an arbitrary video path using MMAction2

# Replace with your video path
video_path = '/content/mmaction2/heatmap_video.mp4'

# If MMAction2 was installed from source:
def mmaction2_inference_from_source(video_path):
    # Running the inference command directly
    !python demo/demo_inferencer.py $video_path --rec tsn --print-result --label-file tools/data/kinetics/label_map_k400.txt

# If MMAction2 was installed as a Python package:
def mmaction2_inference_from_package(video_path):
    from operator import itemgetter
    from mmaction.apis import init_recognizer, inference_recognizer

    # Updated paths based on the provided output
    config_file = '/content/mmaction2/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb.py'
    checkpoint_file = '/content/mmaction2/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth'
    label_file = 'tools/data/kinetics/label_map_k400.txt'
    model = init_recognizer(config_file, checkpoint_file, device='cuda:0')  # assuming GPU is available
    pred_result = inference_recognizer(model, video_path)

    pred_scores = pred_result['predictions'][0]['rec_scores'][0][0].tolist()
    score_tuples = tuple(zip(range(len(pred_scores)), pred_scores))
    score_sorted = sorted(score_tuples, key=itemgetter(1), reverse=True)
    top5_label = score_sorted[:5]

    labels = open(label_file).readlines()
    labels = [x.strip() for x in labels]
    results = [(labels[k[0]], k[1]) for k in top5_label]

    print('The top-5 labels with corresponding scores are:')
    for result in results:
        print(f'{result[0]}: ', result[1])

# Uncomment the relevant function based on your installation method:
mmaction2_inference_from_source(video_path)
# mmaction2_inference_from_package(video_path)


Loads checkpoint by http backend from path: https://download.openmmlab.com/mmaction/v1.0/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth
Downloading: "https://download.openmmlab.com/mmaction/v1.0/recognition/tsn/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth" to /root/.cache/torch/hub/checkpoints/tsn_imagenet-pretrained-r50_8xb32-1x1x8-100e_kinetics400-rgb_20220906-2692d16c.pth
100% 93.1M/93.1M [00:00<00:00, 166MB/s]
/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmaction.visualization.video_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '
Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   09/07 18:47:36 - mmengine - WARNING - "FileCl